# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [168]:
import pandas as pd
# Use this cell to write your code for Task 1
df = pd.read_csv("house_sales.csv")
city_names = ['Silvertown', 'Riverford', 'Teasdale', 'Poppleton']

missing_city = len(df.query('city not in @city_names'))

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [169]:
import numpy as np

# Use this cell to write your code for Task 2
df = pd.read_csv("house_sales.csv")
city_names = ['Silvertown', 'Riverford', 'Teasdale', 'Poppleton', 'Unknown']
house_types = ['Detached','Semi-detached','Terraced']

df['city'] = df['city'].str.replace('--','Unknown')
df['city'] = pd.Categorical(df['city'], categories=city_names, ordered=False)

print(df['bedrooms'].describe())
df['sale_date'] = pd.to_datetime(df['sale_date'])


avg_months_listed = round(np.mean(df['months_listed']),1)
df['months_listed'] = df['months_listed'].fillna(avg_months_listed)

df['house_type'] = np.select(
    [df['house_type'].isin(['Det.']), df['house_type'].isin(['Semi']), df['house_type'].isin(['Terr.'])],
    ['Detached', 'Semi-detached', 'Terraced'],
    df['house_type']
)
df['house_type'] = pd.Categorical(df['house_type'], categories=house_types, ordered=True)
df['area'] = df['area'].str.replace('sq.m.','').replace(' ','').astype(float) # fillna(str(avg_mean)+unit_str)

clean_data = df.copy()



count    1500.000000
mean        4.018000
std         1.413391
min         2.000000
25%         3.000000
50%         4.000000
75%         5.000000
max         6.000000
Name: bedrooms, dtype: float64


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [170]:
# Use this cell to write your code for Task 3
df = pd.read_csv("house_sales.csv")
print(df['sale_price'].isna().sum())

price_by_rooms = df[['bedrooms','sale_price']].groupby('bedrooms').agg({'sale_price': [np.mean, np.var]})
price_by_rooms.columns = price_by_rooms.columns.droplevel()
price_by_rooms = price_by_rooms.reset_index()
price_by_rooms['mean'] = price_by_rooms['mean'].round(1)
price_by_rooms['var'] = price_by_rooms['var'].round(1)

price_by_rooms.rename(columns={'mean':'avg_price', 'var':'var_price'},inplace=True)
print(price_by_rooms.head())

0
   bedrooms  avg_price     var_price
0         2    67076.4  5.652896e+08
1         3   154665.1  2.378289e+09
2         4   234704.6  1.725211e+09
3         5   301515.9  2.484328e+09
4         6   375741.3  3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [171]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import LinearSVR


In [172]:
# Use this cell to write your code for Task 4
training_data = pd.read_csv('train.csv')
validation_data = pd.read_csv('validation.csv')

print(training_data['sale_price'].info())

# # One hot encode city and house_type (could do ordinal encording of )
# training_data = pd.get_dummies(training_data, prefix=['city','house_type'], columns=['city','house_type'],drop_first=False)
# validation_data = pd.get_dummies(validation_data, prefix=['city','house_type'], columns=['city','house_type'],drop_first=False)



# One hot encode city
training_data = pd.get_dummies(training_data, prefix='city', columns=['city'],drop_first=False)
validation_data = pd.get_dummies(validation_data, prefix='city', columns=['city'],drop_first=False)



# Ordinal encode house_type 
# FE: THIS MAY BE A REPEAT OF THE SQUARE FOOTAGE DATA
house_types = ['Detached','Semi-detached','Terraced']
for s, num in zip(house_types,['0','1','2']):
    training_data['house_type'] = training_data['house_type'].str.replace(s,num)
    validation_data['house_type'] = training_data['house_type'].str.replace(s,num)
training_data['house_type'] =  training_data['house_type'].astype(int)
validation_data['house_type'] =  validation_data['house_type'].astype(int)




# Splt sale_date, though this may not be relevant to final result
# FE : Could feature engineer this into season and year
training_data['sale_date'] = pd.to_datetime(training_data['sale_date'])
validation_data['sale_date'] = pd.to_datetime(validation_data['sale_date'])

training_data['sale_year'] = training_data['sale_date'].dt.year
validation_data['sale_year'] = validation_data['sale_date'].dt.year

training_data['sale_month'] = training_data['sale_date'].dt.month
validation_data['sale_month'] = validation_data['sale_date'].dt.month

training_data['sale_day'] = training_data['sale_date'].dt.day
validation_data['sale_day'] = validation_data['sale_date'].dt.day

training_data = training_data.drop(columns='sale_date')
validation_data = validation_data.drop(columns='sale_date')



# # Normalize some of the columns, sale price, area,
training_data['area'] = np.log(training_data['area'])
validation_data['area'] = np.log(validation_data['area'])



target_col = 'sale_price'
feature_cols = training_data.drop(columns=['house_id','sale_price']).columns

X_train = training_data[feature_cols].values
y_train = training_data[target_col].values

X_test = validation_data[feature_cols].values
base_result=validation_data[['house_id']]

<class 'pandas.core.series.Series'>
RangeIndex: 1200 entries, 0 to 1199
Series name: sale_price
Non-Null Count  Dtype
--------------  -----
1200 non-null   int64
dtypes: int64(1)
memory usage: 9.5 KB
None


## RandomForestRegressor GridSearchCV

In [160]:

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt', 'log2']
}

rfr = RandomForestRegressor(random_state=42)

# Setup GridSearchCV
grid_search = GridSearchCV(
    estimator=rfr,
    param_grid=param_grid,
    cv=5,
    scoring= 'neg_root_mean_squared_error',  # mse_scorer, 
    n_jobs=-1,
    verbose=1
)


grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best CV RMSE:", -grid_search.best_score_)

# Best model (can be used to predict on validation set)
best_rfr = grid_search.best_estimator_

Fitting 5 folds for each of 72 candidates, totalling 360 fits


In [163]:

y_test = best_rfr.predict(X_test)

print(rmse(y_train,  np.round(best_rfr.predict(X_train))))

base_result['price'] = y_test 
base_result['price'] = base_result['price'].astype(int)

base_result.head()

8032.5673797360705


,house_id,price
0,1331375,161644
1,1630115,308907
2,1645745,402235
3,1336775,148754
4,1888274,264521


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

## GradientBoostingRegressor GridSearchCV

In [164]:
param_grid = {
    'n_estimators': [500],             
    'learning_rate': [0.01],      
    'max_depth': [5],                        
    'min_samples_split': [20, 40, 80],              
    'min_samples_leaf': [3],                 
    'subsample': [ 0.8],                 
    'max_features': [None]         
}

gradboost = GradientBoostingRegressor(random_state=78)
grid_search = GridSearchCV(
    estimator=gradboost,
    param_grid=param_grid,
    cv=5,
    scoring= 'neg_root_mean_squared_error',  # mse_scorer, 
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best CV RMSE:", -grid_search.best_score_)

# Best model (can be used to predict on validation set)
best_gbr = grid_search.best_estimator_


Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best parameters: {'learning_rate': 0.01, 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 80, 'n_estimators': 500, 'subsample': 0.8}
Best CV RMSE: 14250.519818009645


In [165]:
# best_gbr_params = {'learning_rate': 0.01, 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 40, 'n_estimators': 500, 'subsample': 0.8}
# best_gbr = GradientBoostingRegressor(**best_gbr_params)
# best_gbr.fit(X_train, y_train)


y_test = np.round(best_gbr.predict(X_test))
compare_result = base_result.copy()

print(rmse(y_train,  np.round(best_gbr.predict(X_train))))

compare_result['price'] = y_test
compare_result['price'] = compare_result['price'].astype(int)

compare_result.head()

11550.590508569969


,house_id,price
0,1331375,157269
1,1630115,307761
2,1645745,395389
3,1336775,144238
4,1888274,266294


# Final Results

In [166]:
base_result.head()

,house_id,price
0,1331375,161644
1,1630115,308907
2,1645745,402235
3,1336775,148754
4,1888274,264521


In [167]:
compare_result.head()

,house_id,price
0,1331375,157269
1,1630115,307761
2,1645745,395389
3,1336775,144238
4,1888274,266294
